In [1]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [8]:
data = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

In [9]:
#adding covariates race, smsa, married, region-of-residence

j =pd.get_dummies(data['yob'])

h = data['qob'].astype(str)+data['state'].astype(str)
hi = pd.get_dummies(h)

m = data['yob'].astype(str)+data['qob'].astype(str)
l = pd.get_dummies(m)

n = pd.get_dummies(data['region'])

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])

h = data['qob'].astype(str)+data['state'].astype(str)
hi = pd.get_dummies(h)

used = list(set(list(h)))
opened = []
for thing in used:
    if thing.startswith('3')==True:
        opened.append(thing)

#dropping linear combinations
his = hi.drop(columns = opened)


#print(np.array(data['black']).T.shape)

b = np.r_[a,np.array(j).T, np.array([list(data['black'])]), np.array([list(data['smsa'])]), np.array([list(data['married'])]) ,np.array(n).T, np.array(his).T]

beta_ols = np.linalg.solve(b@b.T, b@d.T)

#note it's pretty good:
print(beta_ols)

[[ 6.28849135e-02]
 [-3.64609202e-01]
 [-3.68666503e-01]
 [-3.72776964e-01]
 [-3.72812095e-01]
 [-3.75527675e-01]
 [-3.85326442e-01]
 [-3.84674112e-01]
 [-3.87373142e-01]
 [-3.86943465e-01]
 [-3.95277571e-01]
 [-2.54813771e-01]
 [ 1.72223736e-01]
 [ 2.48592535e-01]
 [ 5.21549946e+00]
 [ 5.23264694e+00]
 [ 5.04269547e+00]
 [ 5.15521802e+00]
 [ 5.12274876e+00]
 [ 5.12035875e+00]
 [ 5.08164071e+00]
 [ 5.09989625e+00]
 [ 5.11185399e+00]
 [ 4.28078768e-03]
 [ 1.92203173e-02]
 [ 7.84682024e-02]
 [-5.54277403e-02]
 [-2.06439879e-02]
 [-3.91937995e-02]
 [ 2.53842532e-02]
 [ 7.78068538e-03]
 [-3.06528603e-02]
 [-1.45137713e-02]
 [ 2.44175419e-01]
 [-2.74052790e-02]
 [ 1.66357349e-02]
 [ 4.76769474e-02]
 [-1.26185429e-01]
 [ 6.20342397e-02]
 [-1.48499122e-02]
 [ 2.37255008e-02]
 [ 6.07490587e-03]
 [-1.52997850e-02]
 [ 1.59654939e-02]
 [-1.94320325e-02]
 [-2.81263331e-02]
 [ 1.65528039e-01]
 [-7.94042626e-02]
 [ 2.56270671e-02]
 [-5.76940292e-02]
 [ 2.22214271e-02]
 [-6.83117010e-02]
 [ 6.4515815

In [21]:
data1 = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

oo = 0
for i in range(20):
    #my computer can't compute the whole projection matrix so here's some sampling lol
    data = data1.sample(n=10000)
    
    #creating dummies
    j =pd.get_dummies(data['yob'])
    k =pd.get_dummies(data['qob'])
    n = pd.get_dummies(data['region'])
    
    h = data['qob'].astype(str)+data['state'].astype(str)
    hi = pd.get_dummies(h)

    m = data['yob'].astype(str)+data['qob'].astype(str)

    l = pd.get_dummies(m)
    used = list(set(list(h)))
    opened = []
    for thing in used:
        if thing.startswith('3')==True:
            opened.append(thing)
        elif thing in ['21', '11', '41'] == True:
            opened.append(thing)
    
    #dropping linear combinations
    his = hi.drop(columns = opened)
    
    used = list(set(list(m)))
    opened = []
    for thing in used:
        if thing.endswith('3')==True:
            opened.append(thing)
        elif thing.startswith('1939')== True:
            opened.append(thing)
    
    l = l.drop(columns = opened)

    j = j.drop(columns = [1939])

    k = k.drop(columns = [3])


    #creating matrices
    y = np.array([list(data['logwage'])]).squeeze()
    a = np.array([list(data['edu'])])

    x = np.r_[np.array([list(data['edu'])]), np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T,
             np.array(his).T].squeeze()
    
    

    #computation heavy stuff
    z = np.r_[np.array(l).T, np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T, np.array(his).T]
    z = z.astype('float32')
    zint = np.linalg.pinv(z@z.T)
    
    #checking with canned functions
    model = sm.OLS(a.T,z.T)
    results = model.fit()
    reses = results.params
    
    xhat = reses@z
    
    xs = np.r_[[xhat.T], np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T, np.array(his).T]
    model = sm.OLS(y.T,xs.T)
    results = model.fit()
    print(results.params[0])
    #canned gives same answers as both paper and samples

    #the really computation-heavy stuff, according to where my computer chooses to die most frequently
    zpr = z.T@zint
    zproj = zpr@z
    xzpr =x@zproj
    xzprx = xzpr@x.T
    zpry = xzpr@y.T
    beta_iv = np.linalg.solve(xzprx, zpry)
    beta_iv_x = beta_iv
    oo= oo+beta_iv_x[0]
    print(beta_iv_x[0])
print('average')
print(oo/20)

0.07204933210951409
0.07204920839170664
0.08383170319826555
0.08383183898824749
0.11037696524524745
0.11037723464395834
0.04903875430118171
0.04903890100210006
0.08108222919387634
0.08108220933369016
0.09137482903494623
0.09137477187234976
0.07244290920039512
0.07244310860454571
0.10761301699479744
0.10761289814208053
0.04354202418548782
0.04354197661154745
0.09468499373172268
0.09468475217089761
0.02673795602231885
0.02673798286101778
0.06726065876740517
0.0672606711529989
0.0380374370469615
0.0380374534004632
0.06466594855192609
0.06466571274788491
0.05263348638291093
0.05263364603287551
0.0882551638558598
0.08825519676593306
0.10387804504745521
0.10387807454600534
0.08683567697308076
0.08683583573830031
-0.005289260899850448
-0.005289232796183423
0.11696965225628045
0.1169695737897218
average
0.07230109070000708
